In [ ]:
import numpy as np
import awkward as ak
import uproot, icp, json

from pathlib import Path

np.set_printoptions(precision=6)
np.set_printoptions(suppress=True)


In [ ]:
availableOverlapIDs = []
with open("comparisonData/sensors-1.00/rawData/detectorOverlapsIdeal.json") as f:
    idealOverlaps = json.load(f)
    for overlap in idealOverlaps:
        availableOverlapIDs.append(idealOverlaps[overlap]["overlapID"])


In [ ]:
# read from root and sort to numpy, process one root file ENTTIRELY at a time an then write all np arrays to disk
# only then read next file (saves on IO)

filename = "comparisonData/sensors-1.00/rawData/Lumi_Pairs_*.root"

runIndex = 0
maxNoOfFiles = 10
npyOutputDir = "comparisonData/sensors-1.00/npPairs"

if not Path(npyOutputDir).exists():
    Path(npyOutputDir).mkdir(parents=True)

for arrays in uproot.iterate(
    filename,
    [
        "PndLmdHitPair._overlapID",
        "PndLmdHitPair._hit1",
        "PndLmdHitPair._hit2",
    ],
    # library="np", # DONT use numpy yet, we need the awkward array for the TVector3
    allow_missing=True,  # some files may be empty, skip those):
):
    runIndex += 1

    # some evvents have no hits, but thats no a problem
    # after the arrays are flattened, those empty events 
    # simply disappear
    overlapIDs = np.array(ak.flatten(arrays["PndLmdHitPair._overlapID"]))
    hit1x = ak.flatten(arrays["PndLmdHitPair._hit1"].fX)
    hit1y = ak.flatten(arrays["PndLmdHitPair._hit1"].fY)
    hit1z = ak.flatten(arrays["PndLmdHitPair._hit1"].fZ)
    hit2x = ak.flatten(arrays["PndLmdHitPair._hit2"].fX)
    hit2y = ak.flatten(arrays["PndLmdHitPair._hit2"].fY)
    hit2z = ak.flatten(arrays["PndLmdHitPair._hit2"].fZ)

    hit1 = np.array((hit1x, hit1y, hit1z)).T
    hit2 = np.array((hit2x, hit2y, hit2z)).T

    distVec = np.linalg.norm(hit1 - hit2, axis=1)

    # TODO: don't store ditance
    arr = np.array((overlapIDs, hit1x, hit1y, hit1z, hit2x, hit2y, hit2z, distVec)).T

    # TODO: in the future, sort by moduleID, not overlap
    for overlap in availableOverlapIDs:
        mask = arr[:, 0] == overlap
        thisOverlapsArray = arr[mask][:, 1:]

        # read array from disk
        fileName = f"{npyOutputDir}/pairs-{overlap}.npy"

        try:
            oldContent = np.load(fileName)
        # first run, file not already present
        except:
            oldContent = np.empty((0, 7))

        # merge
        newContent = np.concatenate((oldContent, thisOverlapsArray))

        # write back to disk
        np.save(file=fileName, arr=newContent, allow_pickle=False)

    if runIndex == maxNoOfFiles:
        break


In [ ]:
def dynamicCut(hitPairs, cutPercent=2):

    if cutPercent == 0:
        return hitPairs

    # calculate center of mass of differences
    dRaw = hitPairs[:, 3:6] - hitPairs[:, :3]
    com = np.average(dRaw, axis=0)

    # shift newhit2 by com of differences
    newhit2 = hitPairs[:, 3:6] - com

    # calculate new distance for cut
    dRaw = newhit2 - hitPairs[:, :3]
    newDist = np.power(dRaw[:, 0], 2) + np.power(dRaw[:, 1], 2)

    # sort by distance and cut some percent from end (discard outliers)
    cut = int(len(hitPairs) * cutPercent / 100.0)
    # sort by new distance
    hitPairs = hitPairs[newDist.argsort()]
    # cut off largest distances, NOT lowest
    hitPairs = hitPairs[:-cut]

    return hitPairs


In [ ]:
# read required external data
def loadMatrices(filename):
    with open(filename) as f:
        result = json.load(f)
    for key, value in result.items():
        result[key] = np.array(value).reshape(4, 4)
    return result


In [ ]:
idealDetectorMatrices = loadMatrices(
    "comparisonData/oldInput/detectorMatricesIdeal.json"
)
use2D = True

with open("comparisonData/modules-1.00/rawData/detectorOverlapsIdeal.json") as f:
    idealOverlapInfos = json.load(f)


def findMatrix(thisOverlap):

    PairData = np.load(f"comparisonData/sensors-1.00/npPairs/pairs-{thisOverlap}.npy")
    # apply dynamic cut
    PairData = dynamicCut(PairData, 2)

    # if idealOverlapInfos is None or PairData is None:
    #     raise Exception(f"Error! Please load ideal detector matrices and numpy pairs!")

    # if len(idealDetectorMatrices) < 1:
    #     raise Exception("ERROR! Please set ideal detector matrices!")

    # Make C a homogeneous representation of hits1 and hits2
    hit1H = np.ones((len(PairData), 4))
    hit1H[:, 0:3] = PairData[:, :3]

    hit2H = np.ones((len(PairData), 4))
    hit2H[:, 0:3] = PairData[:, 3:6]

    # Attention! Always transform to module-local system,
    # otherwise numerical errors will make the ICP matrices unusable!
    # (because z is at 11m, while x is 30cm and y is 0)
    # also, we're ignoring z distance, which we can not do if we're in
    # PND global, due to the 40mrad rotation.
    transformToLocalSensor = True
    if transformToLocalSensor:
        icpDimension = 2
        # get matrix lmd to module
        modulePath = idealOverlapInfos[str(thisOverlap)]["pathModule"]
        matToModule = idealDetectorMatrices[modulePath]

        # invert to transform pairs from lmd to sensor
        toModInv = np.linalg.inv(matToModule)

        # Transform vectors (remember, C and D are vectors of vectors = matrices!)
        hit1T = np.matmul(toModInv, hit1H.T).T
        hit2T = np.matmul(toModInv, hit2H.T).T

    else:
        print("WARNING! ICP working in Panda global, NOT sensor local.")
        print("This will likely produce wrong overlap matrices!")
        hit1T = hit1H
        hit2T = hit2H

    if use2D:
        icpDimension = 2
        # make 2D versions for ICP
        A = hit1T[:, :2]
        B = hit2T[:, :2]
    else:
        icpDimension = 3
        # make 3D versions for ICP
        A = hit1T[:, :3]
        B = hit2T[:, :3]

    # find ideal transformation
    T, _, _ = icp.best_fit_transform(A, B)

    # copy 3x3 Matrix to 4x4 Matrix
    if icpDimension == 2:
        M = np.identity(4)
        M[:2, :2] = T[:2, :2]
        M[:2, 3] = T[:2, 2]
        thisOverlapMatrix = M

    elif icpDimension == 3:
        thisOverlapMatrix = T

    transformResultToPND = True
    if transformResultToPND:
        # remember, matToModule goes from Pnd->Module
        # base trafo is T A T^-1,
        # T = Pnd->Module

        # TODO: use generalized base transformation function
        thisOverlapMatrix = (
            (matToModule) @ thisOverlapMatrix @ np.linalg.inv(matToModule)
        )
    return thisOverlapMatrix


In [ ]:
overlapMatrices = {}
for overlap in availableOverlapIDs:
    overlapMatrices[str(overlap)] = findMatrix(overlap)


In [ ]:
for overlap in availableOverlapIDs:

    
    # print(overlapMatrices[str(overlap)] * 1e4)

    mOldOverlap = loadMatrices("comparisonData/sensors-1.00/alMat-sensorOverlaps-1.00.json")
    # print(mOld[str(overlap)] * 1e4)
    mDiff = overlapMatrices[str(overlap)]@np.linalg.inv(mOldOverlap[str(overlap)])
    # print(f'mDiff:\n{mDiff*1e4}')
    # print("--------------------")
    # break

# looking good so far


In [ ]:
from numpy.linalg import inv

"""
Author: R. Klasen, roklasen@uni-mainz.de or r.klasen@gsi.de or r.klasen@ep1.rub.de

Uses multiple overlap matrices and the ideal detector matrices to compute alignment matrices for each sensor.
Each combiner is responsible for a single module.

It requires the misalignment matrices of sensor 0 and 1 for its assigned module.
We will obtain these with microscopic measurements.

The overlap matrices come from the Reco Points and are thus already in PANDA global. All calculations must
happen in PANDA GLOBAL, but the actual misalignment matrices are applied in the current super frame.
That means the misalignment matrix for sensor 1 is applied in the frame of sensor 1, NOT PANDA GLOBAL.
(Doesn't matter how it was generated, this is the way it's done by FAIRROOT, and that's by design).
For the math to work out, all matrices must be transformed to PANDA GLOBAL before calculations.
To compare the found alignment matrices with the input misalignment matrces, the found alignment
matrices must be transformed to the reference frame of the corresponding sensor.
"""


class alignmentMatrixCombiner:
    def __init__(self, modulePath):
        self.modulePath = modulePath
        self.alignmentMatrices = {}
        self.overlapMatrices = None
        self.idealDetectorMatrices = None
        self.externalMatrices = None

    def setOverlapMatrices(self, matrices):
        self.overlapMatrices = matrices

    def setIdealDetectorMatrices(self, matrices):
        self.idealDetectorMatrices = matrices

    def setExternallyMeasuredMatrices(self, matrices):
        self.externalMatrices = matrices

    def getDigit(self, number, n):
        return int(number) // 10**n % 10

    def getSmallOverlap(self, overlap):
        return str(self.getDigit(overlap, 0))

    def getAlignmentMatrices(self):
        return self.alignmentMatrices

    def sortOverlapMatrices(self):
        tempDict = {}
        # add small overlap to each overlap info, we'll need that info later
        for overlapID in self.overlapMatrices:
            smallOverlap = self.getSmallOverlap(overlapID)
            print(
                f"Attention! Small Overlap translation! Was {overlapID}, is now {smallOverlap}"
            )
            tempDict[smallOverlap] = self.overlapMatrices[overlapID]

        # and sort by new small overlap. that's okay, the real overlapID is still a field of the dict
        self.overlapMatrices = tempDict

    def getIdealMatrixP1ToP2(self, path1, path2):
        # matrix from pnd global to sen1
        m1 = self.idealDetectorMatrices[path1]
        # matrix from pnd global to sen2
        m2 = self.idealDetectorMatrices[path2]
        # matrix from sen1 to sen2
        return m2 @ inv(m1)

    def baseTransform(self, mat, matFromAtoB):
        """
        Reminder: the way this works is that the matrix pointing from pnd to sen0 transforms a matrix IN sen0 back to Pnd
        If you want to transform a matrix from Pnd to sen0, and you have the matrix to sen0, then you need to give
        this function inv(matTo0). I know it's confusing, but that's the way this works.

        Example: matrixInPanda = baseTransform(matrixInSensor, matrixPandaToSensor)
        """
        return matFromAtoB @ mat @ inv(matFromAtoB)

    def combine1to2(self):

        p1 = self.modulePath + "/sensor_1"
        p2 = self.modulePath + "/sensor_2"
        p8 = self.modulePath + "/sensor_8"

        m1 = self.idealDetectorMatrices[p1]
        m2 = self.idealDetectorMatrices[p2]

        m1to2Ideal = self.getIdealMatrixP1ToP2(p1, p2)
        m1misInPnd = self.baseTransform(self.externalMatrices[p1], m1)

        mICP1t8 = self.overlapMatrices["4"]
        mICP2t8 = self.overlapMatrices["5"]

        # print("------------------------------------")
        # print(f"m4 is:\n{mICP1t8}")
        # print(f"should be:\n{mOldOverlap['4']}")
        # print("------------------------------------")
        # print(f"m4 is:\n{mICP2t8}")
        # print(f"should be:\n{mOldOverlap['5']}")
        # print("------------------------------------")

        # * this is the actual computation
        m1to2FromICP = inv(inv(mICP2t8) @ mICP1t8) @ m1to2Ideal

        # transform this matrix to the system of misaligned sensor1
        mTotalFromMathInAstar = self.baseTransform(m1to2FromICP, m1misInPnd)
        mBstar = mTotalFromMathInAstar @ m1misInPnd @ inv(m1to2Ideal)

        # transform it to senB, because that's where it lives:
        mBstarInSenB = self.baseTransform(mBstar, inv(m2))
        return mBstarInSenB

    def combine1to2New(self):

        # required paths and ideal matrices for base transformations
        p1 = self.modulePath + "/sensor_1"
        p2 = self.modulePath + "/sensor_2"
        m1 = self.idealDetectorMatrices[p1]
        m2 = self.idealDetectorMatrices[p2]

        # attention! m1star is applied by FAIRROOT in the system of sensor 1
        # that means m1star here is given in sensor 1, not PANDA GLOBAL
        # for the math to work out, m1star must be transformed to PANDA GLOBAL
        m1starInSensor1 = self.externalMatrices[p1]
        m1star = self.baseTransform(m1starInSensor1, m1)

        # these are the overlap matrices J18 and J28
        mICP1t8 = self.overlapMatrices["4"]
        mICP2t8 = self.overlapMatrices["5"]

        # now the math from my thesis checks out
        m2star = m1star @ inv(inv(mICP2t8) @ mICP1t8)

        # now, transform m2star from PANDA GLOBAL into the system of sensor2
        # (because thats where it was applied by FAIRROOT)
        m2starInSensor2 = self.baseTransform(m2star, inv(m2))

        return m2starInSensor2

    def combine1to3(self):

        # * matrix path: mat1t8 -> mat8to3
        # the rest is pretty much like in 1->2

        p1 = self.modulePath + "/sensor_1"
        p3 = self.modulePath + "/sensor_3"
        p8 = self.modulePath + "/sensor_8"

        m1 = self.idealDetectorMatrices[p1]
        m3 = self.idealDetectorMatrices[p3]

        m1to3Ideal = self.getIdealMatrixP1ToP2(p1, p3)
        m1misInPnd = self.baseTransform(self.externalMatrices[p1], m1)

        mICP1t8 = self.overlapMatrices["4"]
        mICP3t8 = self.overlapMatrices["1"]

        # * this is the actual computation
        m1to3FromICP = inv(inv(mICP3t8) @ mICP1t8) @ m1to3Ideal

        # transform this matrix to the system of misaligned sensor1
        m1to3FromICPInAstar = self.baseTransform(m1to3FromICP, m1misInPnd)
        mBstar = m1to3FromICPInAstar @ m1misInPnd @ inv(m1to3Ideal)

        # transform it to senB, because that's where it lives:
        mBstarInSenB = self.baseTransform(mBstar, inv(m3))
        return mBstarInSenB

    def combine1to4a(self):
        # * matrix path: mat1t8 -> mat8to2 -> mat2to9 -> mat9to4

        p1 = self.modulePath + "/sensor_1"
        p2 = self.modulePath + "/sensor_2"
        p4 = self.modulePath + "/sensor_4"
        p8 = self.modulePath + "/sensor_8"
        p9 = self.modulePath + "/sensor_9"

        m1 = self.idealDetectorMatrices[p1]
        m4 = self.idealDetectorMatrices[p4]

        m1to4Ideal = self.getIdealMatrixP1ToP2(p1, p4)
        m1misInPnd = self.baseTransform(self.externalMatrices[p1], m1)

        mICP1t8 = self.overlapMatrices["4"]
        mICP2t8 = self.overlapMatrices["5"]
        mICP2t9 = self.overlapMatrices["6"]
        mICP4t9 = self.overlapMatrices["2"]

        # * this is the actual computation
        m1to4FromICP = inv(inv(mICP4t9) @ mICP2t9 @ inv(mICP2t8) @ mICP1t8) @ m1to4Ideal

        # transform this matrix to the system of misaligned sensor1
        m1to4FromICPInAstar = self.baseTransform(m1to4FromICP, m1misInPnd)
        mBstar = m1to4FromICPInAstar @ m1misInPnd @ inv(m1to4Ideal)

        # transform it to senB, because that's where it lives:
        mBstarInSenB = self.baseTransform(mBstar, inv(m4))
        return mBstarInSenB

    def combine1to4b(self):
        # * matrix path: mat1t8 -> mat8to3 -> mat3to7 -> mat7to4

        p1 = self.modulePath + "/sensor_1"
        p3 = self.modulePath + "/sensor_3"
        p4 = self.modulePath + "/sensor_4"
        p7 = self.modulePath + "/sensor_7"
        p8 = self.modulePath + "/sensor_8"

        m1 = self.idealDetectorMatrices[p1]
        m4 = self.idealDetectorMatrices[p4]

        m1to4Ideal = self.getIdealMatrixP1ToP2(p1, p4)
        m1misInPnd = self.baseTransform(self.externalMatrices[p1], m1)

        mICP1t8 = self.overlapMatrices["4"]
        mICP3t7 = self.overlapMatrices["7"]
        mICP3t8 = self.overlapMatrices["1"]
        mICP4t7 = self.overlapMatrices["8"]

        # * this is the actual computation
        m1to4FromICP = inv(inv(mICP4t7) @ mICP3t7 @ inv(mICP3t8) @ mICP1t8) @ m1to4Ideal

        # transform this matrix to the system of misaligned sensor1
        m1to4FromICPInAstar = self.baseTransform(m1to4FromICP, m1misInPnd)
        mBstar = m1to4FromICPInAstar @ m1misInPnd @ inv(m1to4Ideal)

        # transform it to senB, because that's where it lives:
        mBstarInSenB = self.baseTransform(mBstar, inv(m4))
        return mBstarInSenB

    def combine0to5(self):
        # * matrix path: mat0t5

        p0 = self.modulePath + "/sensor_0"
        p5 = self.modulePath + "/sensor_5"

        m0 = self.idealDetectorMatrices[p0]
        m5 = self.idealDetectorMatrices[p5]

        m0to5Ideal = self.getIdealMatrixP1ToP2(p0, p5)
        m0misInPnd = self.baseTransform(self.externalMatrices[p0], m0)

        mICP0t5 = self.overlapMatrices["0"]

        # * this is the actual computation
        m0to5FromICP = inv(mICP0t5) @ m0to5Ideal

        # transform this matrix to the system of misaligned sensor0
        m0to5FromICPInAstar = self.baseTransform(m0to5FromICP, m0misInPnd)
        mBstar = m0to5FromICPInAstar @ m0misInPnd @ inv(m0to5Ideal)

        # transform it to senB, because that's where it lives:
        mBstarInSenB = self.baseTransform(mBstar, inv(m5))
        return mBstarInSenB

    def combine1to6(self):
        # * matrix path: mat1t8 -> mat8to3 -> mat3to6

        p1 = self.modulePath + "/sensor_1"
        p3 = self.modulePath + "/sensor_3"
        p6 = self.modulePath + "/sensor_6"
        p8 = self.modulePath + "/sensor_8"

        m1 = self.idealDetectorMatrices[p1]
        m6 = self.idealDetectorMatrices[p6]

        m1to6Ideal = self.getIdealMatrixP1ToP2(p1, p6)
        m1misInPnd = self.baseTransform(self.externalMatrices[p1], m1)

        mICP1t8 = self.overlapMatrices["4"]
        mICP3t6 = self.overlapMatrices["3"]
        mICP3t8 = self.overlapMatrices["1"]

        # * this is the actual computation
        m1to6FromICP = inv(mICP3t6 @ inv(mICP3t8) @ mICP1t8) @ m1to6Ideal

        # transform this matrix to the system of misaligned sensor1
        m1to6FromICPInAstar = self.baseTransform(m1to6FromICP, m1misInPnd)
        mBstar = m1to6FromICPInAstar @ m1misInPnd @ inv(m1to6Ideal)

        # transform it to senB, because that's where it lives:
        mBstarInSenB = self.baseTransform(mBstar, inv(m6))
        return mBstarInSenB

    def combine1to7a(self):
        # * matrix path: mat1t8 -> mat8to3 -> mat3to7

        p1 = self.modulePath + "/sensor_1"
        p3 = self.modulePath + "/sensor_3"
        p7 = self.modulePath + "/sensor_7"
        p8 = self.modulePath + "/sensor_8"

        m1 = self.idealDetectorMatrices[p1]
        m7 = self.idealDetectorMatrices[p7]

        m1to7Ideal = self.getIdealMatrixP1ToP2(p1, p7)
        m1misInPnd = self.baseTransform(self.externalMatrices[p1], m1)

        mICP1t8 = self.overlapMatrices["4"]
        mICP3t8 = self.overlapMatrices["1"]
        mICP3t7 = self.overlapMatrices["7"]

        # * this is the actual computation
        m1to7FromICP = inv(mICP3t7 @ inv(mICP3t8) @ mICP1t8) @ m1to7Ideal

        # transform this matrix to the system of misaligned sensor1
        m1to7FromICPInAstar = self.baseTransform(m1to7FromICP, m1misInPnd)
        mBstar = m1to7FromICPInAstar @ m1misInPnd @ inv(m1to7Ideal)

        # transform it to senB, because that's where it lives:
        mBstarInSenB = self.baseTransform(mBstar, inv(m7))
        return mBstarInSenB

    def combine1to7b(self):
        # * matrix path: mat1t8 -> mat8to2 -> mat2to9 -> mat9to4 -> mat4to7

        p1 = self.modulePath + "/sensor_1"
        p2 = self.modulePath + "/sensor_2"
        p4 = self.modulePath + "/sensor_4"
        p7 = self.modulePath + "/sensor_7"
        p8 = self.modulePath + "/sensor_8"
        p9 = self.modulePath + "/sensor_9"

        m1 = self.idealDetectorMatrices[p1]
        m7 = self.idealDetectorMatrices[p7]

        m1to7Ideal = self.getIdealMatrixP1ToP2(p1, p7)
        m1misInPnd = self.baseTransform(self.externalMatrices[p1], m1)

        mICP1t8 = self.overlapMatrices["4"]
        mICP2t8 = self.overlapMatrices["5"]
        mICP2t9 = self.overlapMatrices["6"]
        mICP4t7 = self.overlapMatrices["8"]
        mICP4t9 = self.overlapMatrices["2"]

        # * this is the actual computation
        m1to7FromICP = (
            inv(mICP4t7 @ inv(mICP4t9) @ mICP2t9 @ inv(mICP2t8) @ mICP1t8) @ m1to7Ideal
        )

        # transform this matrix to the system of misaligned sensor1
        m1to7FromICPInAstar = self.baseTransform(m1to7FromICP, m1misInPnd)
        mBstar = m1to7FromICPInAstar @ m1misInPnd @ inv(m1to7Ideal)

        # transform it to senB, because that's where it lives:
        mBstarInSenB = self.baseTransform(mBstar, inv(m7))
        return mBstarInSenB

    def combine1to8(self):
        # * matrix path: mat1t8 -> mat8to3 -> mat3to6

        p1 = self.modulePath + "/sensor_1"
        p8 = self.modulePath + "/sensor_8"

        m1 = self.idealDetectorMatrices[p1]
        m8 = self.idealDetectorMatrices[p8]

        m1to8Ideal = self.getIdealMatrixP1ToP2(p1, p8)
        m1misInPnd = self.baseTransform(self.externalMatrices[p1], m1)

        # mICP1t8 = self.getOverlapMisalignLikeICP(p1, p8)

        mICP1t8 = self.overlapMatrices["4"]

        # * this is the actual computation
        m1to8FromICP = inv(mICP1t8) @ m1to8Ideal

        # transform this matrix to the system of misaligned sensor1
        m1to8FromICPInAstar = self.baseTransform(m1to8FromICP, m1misInPnd)
        mBstar = m1to8FromICPInAstar @ m1misInPnd @ inv(m1to8Ideal)

        # transform it to senB, because that's where it lives:
        mBstarInSenB = self.baseTransform(mBstar, inv(m8))
        return mBstarInSenB

    def combine1to9a(self):
        # * matrix path: mat1t8 -> mat8to2 -> mat2to9

        p1 = self.modulePath + "/sensor_1"
        p2 = self.modulePath + "/sensor_2"
        p8 = self.modulePath + "/sensor_8"
        p9 = self.modulePath + "/sensor_9"

        m1 = self.idealDetectorMatrices[p1]
        m9 = self.idealDetectorMatrices[p9]

        m1to9Ideal = self.getIdealMatrixP1ToP2(p1, p9)
        m1misInPnd = self.baseTransform(self.externalMatrices[p1], m1)

        mICP1t8 = self.overlapMatrices["4"]
        mICP2t8 = self.overlapMatrices["5"]
        mICP2t9 = self.overlapMatrices["6"]

        # * this is the actual computation
        m1to9FromICP = inv(mICP2t9 @ inv(mICP2t8) @ mICP1t8) @ m1to9Ideal

        # transform this matrix to the system of misaligned sensor1
        m1to9FromICPInAstar = self.baseTransform(m1to9FromICP, m1misInPnd)
        mBstar = m1to9FromICPInAstar @ m1misInPnd @ inv(m1to9Ideal)

        # transform it to senB, because that's where it lives:
        mBstarInSenB = self.baseTransform(mBstar, inv(m9))
        return mBstarInSenB

    def combine1to9b(self):
        # * matrix path: mat1t8 -> mat8to3 -> mat3to7 -> mat7to4 -> mat4to9

        p1 = self.modulePath + "/sensor_1"
        p3 = self.modulePath + "/sensor_3"
        p4 = self.modulePath + "/sensor_4"
        p7 = self.modulePath + "/sensor_7"
        p8 = self.modulePath + "/sensor_8"
        p9 = self.modulePath + "/sensor_9"

        m1 = self.idealDetectorMatrices[p1]
        m9 = self.idealDetectorMatrices[p9]

        m1to9Ideal = self.getIdealMatrixP1ToP2(p1, p9)
        m1misInPnd = self.baseTransform(self.externalMatrices[p1], m1)

        mICP1t8 = self.overlapMatrices["4"]
        mICP3t7 = self.overlapMatrices["7"]
        mICP3t8 = self.overlapMatrices["1"]
        mICP4t7 = self.overlapMatrices["8"]
        mICP4t9 = self.overlapMatrices["2"]

        # * this is the actual computation
        m1to9FromICP = (
            inv(mICP4t9 @ inv(mICP4t7) @ mICP3t7 @ inv(mICP3t8) @ mICP1t8) @ m1to9Ideal
        )

        # transform this matrix to the system of misaligned sensor1
        m1to9FromICPInAstar = self.baseTransform(m1to9FromICP, m1misInPnd)
        mBstar = m1to9FromICPInAstar @ m1misInPnd @ inv(m1to9Ideal)

        # transform it to senB, because that's where it lives:
        mBstarInSenB = self.baseTransform(mBstar, inv(m9))
        return mBstarInSenB

    def combineMatrices(self):
        # checks here
        if self.overlapMatrices is None or self.idealDetectorMatrices is None:
            raise Exception(
                f"ERROR! Please set overlaps, overlap matrices and ideal detector matrices first!"
            )

        if self.externalMatrices is None:
            raise Exception(
                f"ERROR! Please set the externally measured matrices for sensor 0 and 1 for module {self.modulePath}!"
            )

        # the overlap matrices from the matrix finders still have their entire overlapID, but we need only the smallOverlap
        #! ATTENTION!!! This is currently broken if you supply ALL overlap matrices
        # because 8, 18, 38, 108 all map to 8!
        # self.sortOverlapMatrices()

        mat2mis = self.combine1to2()
        print(f"oye! mat2mis old:\n{mat2mis*1e4}")
        print(f"oye! mat2mis new:\n{self.combine1to2New()*1e4}")
        mat3mis = self.combine1to3()
        mat4amis = self.combine1to4a()
        mat4bmis = self.combine1to4b()
        mat5mis = self.combine0to5()
        mat6mis = self.combine1to6()
        mat7amis = self.combine1to7a()
        mat7bmis = self.combine1to7b()
        mat8mis = self.combine1to8()
        mat9amis = self.combine1to9a()
        mat9bmis = self.combine1to9b()

        # compute averages for the matrices that can be reached two ways
        mat4mis = (mat4amis + mat4bmis) / 2  # man I love numpy
        mat7mis = (mat7amis + mat7bmis) / 2
        mat9mis = (mat9amis + mat9bmis) / 2

        # now, all the overlap misalignments are in PND global!

        # copy the given misalignments, so that all are in the final set!
        self.alignmentMatrices[self.modulePath + "/sensor_0"] = self.externalMatrices[
            self.modulePath + "/sensor_0"
        ]
        self.alignmentMatrices[self.modulePath + "/sensor_1"] = self.externalMatrices[
            self.modulePath + "/sensor_1"
        ]

        # store computed misalignment matrices to internal dict
        self.alignmentMatrices[self.modulePath + "/sensor_2"] = mat2mis
        self.alignmentMatrices[self.modulePath + "/sensor_3"] = mat3mis
        self.alignmentMatrices[self.modulePath + "/sensor_4"] = mat4mis
        self.alignmentMatrices[self.modulePath + "/sensor_5"] = mat5mis
        self.alignmentMatrices[self.modulePath + "/sensor_6"] = mat6mis
        self.alignmentMatrices[self.modulePath + "/sensor_7"] = mat7mis
        self.alignmentMatrices[self.modulePath + "/sensor_8"] = mat8mis
        self.alignmentMatrices[self.modulePath + "/sensor_9"] = mat9mis

        print(
            f"successfully computed misalignments on module {self.modulePath} for {len(self.alignmentMatrices)} sensors!"
        )


In [ ]:
externalMatrices = loadMatrices(
    "comparisonData/oldInput/sensorAligner/externalMatrices-sensors-1.00.json"
)

modulePath = idealOverlapInfos[str(0)]["pathModule"]

combiner = alignmentMatrixCombiner(modulePath)
combiner.setIdealDetectorMatrices(idealDetectorMatrices)
combiner.setOverlapMatrices(overlapMatrices)
# combiner.setOverlapMatrices(mOldOverlap)
combiner.setExternallyMeasuredMatrices(externalMatrices)
combiner.combineMatrices()
sensorAlignMatrices = combiner.getAlignmentMatrices()


In [ ]:
testSensor = "/cave_1/lmd_root_0/half_0/plane_0/module_0/sensor_9"

mMat = loadMatrices("comparisonData/misMatrices/misMat-sensors-1.00.json")

mSensorOld = loadMatrices("comparisonData/sensors-1.00/alMat-sensorAlignment-1.00.json")

print("-------------------- this time --------------------")
print(sensorAlignMatrices[testSensor] * 1e4)
print("-------------------- was earlier --------------------")
print(mSensorOld[testSensor] * 1e4)
print("-------------------- should be --------------------")
print(mMat[testSensor] * 1e4)
